In [ ]:
!pip install rouge

In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\tabbi\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\tabbi\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\tabbi\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [ ]:
import random
import heapq
import nltk
from rouge import Rouge
from nltk.corpus import stopwords as nltk_stopwords


In [ ]:

import pandas as pd
import random


In [ ]:

nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\tabbi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tabbi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.chunk import ne_chunk


In [ ]:

# Internal function to calculate Semantic Similarity Factor (SSF)
def calculate_ssf(input_text, generated_summary):
    # Tokenize input text and generated summary
    input_sentences = sent_tokenize(input_text)
    summary_sentences = sent_tokenize(generated_summary)

    # Vectorize input text and generated summary using TF-IDF
    vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = vectorizer.fit_transform(input_sentences + summary_sentences)

    # Calculate cosine similarity between input text and generated summary
    ssf_score = cosine_similarity(tfidf_matrix[:len(input_sentences)], tfidf_matrix[len(input_sentences):]).mean()

    return ssf_score


In [ ]:
def calculate_rlf(input_text, generated_summary, lambda_val=0.25):
    # Tokenize input text and generated summary into lists of words
    input_words = word_tokenize(input_text)
    summary_words = word_tokenize(generated_summary)

    # Calculate number of matching unigrams between reference and summary
    matching_unigrams_ref = len(set(input_words) & set(summary_words))

    # Calculate total number of unigrams in reference and summary
    total_unigrams_ref = len(input_words)
    total_unigrams_sys = len(summary_words)

    # Calculate Precision (P) and Recall (R)
    P = matching_unigrams_ref / total_unigrams_ref
    R = matching_unigrams_ref / total_unigrams_sys

    # Calculate F-score (F)
    F = (10 * P * R) / (R + (9 * P))

    # Calculate number of chunks in the machine-generated summary
    num_chunks = len(set(nltk.ngrams(summary_words, 2)))

    # Calculate Chunk Penalty (CP)
    CP = lambda_val * ((num_chunks / total_unigrams_sys) ** 3)

    # Calculate Relevance Factor (RLF)
    relevance_factor = F * (1 - CP)

    return relevance_factor


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import sent_tokenize

def calculate_cosine_similarity(sentence1, sentence2):
    # Tokenize sentences
    sentences = [sentence1, sentence2]

    # Create TF-IDF vectors
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(sentences)

    # Calculate cosine similarity
    cosine_sim = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])

    return cosine_sim[0][0]


In [ ]:
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Internal function to calculate Redundancy Factor (RDF)
def calculate_rdf(generated_summary):
    # Tokenize the summary into sentences
    sentences = nltk.sent_tokenize(generated_summary)
    num_sentences = len(sentences)

    # Calculate total possible pairs of sentences
    total_pairs = num_sentences * (num_sentences - 1) / 2

    if total_pairs == 0:
        return 0  # Return 0 if there are no pairs of sentences

    # Initialize the count of less redundant pairs
    less_redundant_count = 0

    # Calculate cosine similarity between each pair of sentences
    for i in range(num_sentences):
        for j in range(i + 1, num_sentences):
            # Calculate cosine similarity score between sentence i and sentence j
            similarity_score = calculate_cosine_similarity(sentences[i], sentences[j])
            if similarity_score < 0.5:
                less_redundant_count += 1

    # Calculate the redundancy score (RDF)
    rdf_score = less_redundant_count / total_pairs

    return rdf_score


In [ ]:
!pip install scikit-learn

In [ ]:
from nltk.tokenize import word_tokenize
from nltk import pos_tag, ne_chunk

from nltk.tokenize import word_tokenize

# Internal function to calculate Jaccard Similarity
def jaccard_similarity(set1, set2):
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union
'''
# Internal function to perform Named Entity Recognition (NER)
def calculate_baa(input_text, generated_summary):
    # Tokenize input text and generated summary
    input_tokens = set(word_tokenize(input_text))
    generated_tokens = set(word_tokenize(generated_summary))

    # Calculate Jaccard Similarity score
    baa_score = jaccard_similarity(input_tokens, generated_tokens)

    return baa_score
'''
# Internal function to perform Named Entity Recognition (NER)
def calculate_baa(input_text, generated_summary):
    # Tokenize input text and generated summary
    input_tokens = word_tokenize(input_text)
    generated_tokens = word_tokenize(generated_summary)

    # Perform POS tagging
    input_pos = pos_tag(input_tokens)
    generated_pos = pos_tag(generated_tokens)

    # Perform Named Entity Recognition (NER)
    input_ner = ne_chunk(input_pos)
    generated_ner = ne_chunk(generated_pos)

    # Extract named entities
    input_entities = set()
    for chunk in input_ner:
        if hasattr(chunk, 'label'):
            input_entities.add(' '.join(c[0] for c in chunk))

    generated_entities = set()
    for chunk in generated_ner:
        if hasattr(chunk, 'label'):
            generated_entities.add(' '.join(c[0] for c in chunk))

    # Calculate Jaccard Similarity score
    baa_score = jaccard_similarity(input_entities, generated_entities)

    return baa_score


In [ ]:

def evaluate_fitness( generated_summary,input_text):
    # Calculate Semantic Similarity Factor (SSF)
    ssf_score = calculate_ssf(input_text, generated_summary)

    # Calculate Relevance Factor (RLF)
    rlf_score = calculate_rlf(input_text, generated_summary)

    # Calculate Redundancy Factor (RDF)
    rdf_score = calculate_rdf(generated_summary)

    # Calculate Bias Avoidance Analysis (BAA)
    baa_score = calculate_baa(input_text, generated_summary)

    # Define weights for each factor
    w1 = w2 = w3 = w4 = 0.25

    # Calculate SUSWIR score
    suswir_score = w1 * ssf_score + w2 * rlf_score + w3 * rdf_score + w4 * baa_score

    return suswir_score

In [ ]:
'''
# Objective function
def evaluate_fitness(summary, original_text):
    if summary.strip() == "":
        return 0.0
    else:
        rouge = Rouge()
        scores = rouge.get_scores(summary, original_text)
        return scores[0]["rouge-l"]["f"]
'''

'\n# Objective function\ndef evaluate_fitness(summary, original_text):\n    if summary.strip() == "":\n        return 0.0\n    else:\n        rouge = Rouge()\n        scores = rouge.get_scores(summary, original_text)\n        return scores[0]["rouge-l"]["f"]\n'

In [ ]:
import nltk
import random

def population_initialize(original_text, pop_size):
    # Tokenize the original text into sentences
    tokenized_sentences = nltk.sent_tokenize(original_text)

    # Initialize the population list
    population = []

    # Iterate for the specified population size
    for _ in range(pop_size):
        # Create a copy of the tokenized sentences to keep track of which sentences have been used
        remaining_sentences = tokenized_sentences.copy()

        # Initialize the summary for this iteration
        summary = []

        # Run the loop for len(tokenized_sentences)//2 times
        for _ in range(len(tokenized_sentences) // 2):
            # Randomly select a sentence from the remaining sentences
            selected_sentence = random.choice(remaining_sentences)

            # Add the selected sentence to the summary
            summary.append(selected_sentence)

            # Remove the selected sentence from the list of remaining sentences
            remaining_sentences.remove(selected_sentence)

        # Reorder the summary to match the order of occurrence of sentences in the original text
        reordered_summary = []
        for sentence in tokenized_sentences:
            if sentence in summary:
                reordered_summary.append(sentence)

        # Convert the reordered summary into a string
        reordered_summary_string = ' '.join(reordered_summary)

        # Add the reordered summary string to the population
        population.append(reordered_summary_string)

    return population


In [ ]:

# Selection
def selection(population, original_text):
    fitness_scores = [
        (evaluate_fitness(summary, original_text), summary) for summary in population
    ]
    top_summaries = heapq.nlargest(len(population) // 2, fitness_scores)
    return [summary for _, summary in top_summaries]


In [ ]:
import nltk
import random
'''
def crossover(parent1, parent2):
    # Tokenize sentences in parent1 and parent2
    sentences_1 = nltk.sent_tokenize(parent1)
    sentences_2 = nltk.sent_tokenize(parent2)

    # Calculate the number of sentences in each parent
    num_sentences = min(len(sentences_1), len(sentences_2))
    # Calculate the midpoint for crossover
    crossover_point = num_sentences // 2

    # Create child 1 by combining the first half of sentences from parent1
    # with the second half of sentences from parent2
    child_1 = " ".join(sentences_1[:crossover_point] + sentences_2[crossover_point:])

    # Create child 2 by combining the first half of sentences from parent2
    # with the second half of sentences from parent1
    child_2 = " ".join(sentences_2[:crossover_point] + sentences_1[crossover_point:])

    return child_1, child_2
'''

from nltk.tokenize import sent_tokenize

def crossover(text1, text2):
    # Tokenize texts into sentences
    sentences1 = sent_tokenize(text1)
    sentences2 = sent_tokenize(text2)

    # Randomly select crossover points
    crossover_point1 = random.randint(0, len(sentences1) - 1)
    crossover_point2 = random.randint(0, len(sentences2) - 1)

    # Perform crossover
    crossover_text1 = ' '.join(sentences1[:crossover_point1] + sentences2[crossover_point1:])
    crossover_text2 = ' '.join(sentences2[:crossover_point2] + sentences1[crossover_point2:])

    return crossover_text1, crossover_text2



In [ ]:
'''
# Mutation
def perform_mutation(summary, input_text):
    sentences = nltk.sent_tokenize(input_text)
    if len(sentences) == 0:
        return input_text

    if random.random() < 0.5:
        if len(sentences) > 1:
            sentences.pop(random.randint(0, len(sentences) - 1))
    else:
        sentences.insert(random.randint(0, len(sentences)), random.choice(sentences))
    reordered=[]
    for sentence in input_text:
            if sentence in sentences:
                reordered.append(sentence)
    stopwords = nltk.corpus.stopwords.words("english")
    mutated_summary = " ".join(reordered)
    mutated_summary_words = nltk.word_tokenize(mutated_summary)
    if len(mutated_summary_words) == 0 or all(
        word in stopwords for word in mutated_summary_words
    ):
        return summary

    return mutated_summary
'''

'\n# Mutation\ndef perform_mutation(summary, input_text):\n    sentences = nltk.sent_tokenize(input_text)\n    if len(sentences) == 0:\n        return input_text\n\n    if random.random() < 0.5:\n        if len(sentences) > 1:\n            sentences.pop(random.randint(0, len(sentences) - 1))\n    else:\n        sentences.insert(random.randint(0, len(sentences)), random.choice(sentences))\n    reordered=[]\n    for sentence in input_text:\n            if sentence in sentences:\n                reordered.append(sentence)\n    stopwords = nltk.corpus.stopwords.words("english")\n    mutated_summary = " ".join(reordered)\n    mutated_summary_words = nltk.word_tokenize(mutated_summary)\n    if len(mutated_summary_words) == 0 or all(\n        word in stopwords for word in mutated_summary_words\n    ):\n        return summary\n\n    return mutated_summary\n'

In [ ]:
'''
import random
import nltk
import random
import nltk
def perform_mutation(summary, input_text):
    # Tokenize sentences in input text
    sentences = nltk.sent_tokenize(input_text)

    if len(sentences) == 0:
        return input_text

    if random.random() < 0.5:
        if len(sentences) > 1:
            sentences.pop(random.randint(0, len(sentences) - 1))
    else:
        # Remove common sentences
        common_sentences = set(nltk.sent_tokenize(summary))
        valid_sentences = [s for s in sentences if s not in common_sentences]

        # If there are no valid sentences, return the original summary
        if not valid_sentences:
            return summary

        # Calculate cosine similarity with each valid sentence
        max_similarity = -1
        selected_sentence = None
        for new_sentence in valid_sentences:
            similarity_sum = 0
            for existing_sentence in nltk.sent_tokenize(summary):
                similarity = calculate_cosine_similarity(existing_sentence, new_sentence)
                similarity_sum += similarity
            average_similarity = similarity_sum / len(nltk.sent_tokenize(summary))

            # Update selected sentence if it has the lowest average similarity
            if average_similarity > max_similarity:
                max_similarity = average_similarity
                selected_sentence = new_sentence

        # Insert the selected sentence into the summary
        summary += " " + selected_sentence

    return summary
'''

'\nimport random\nimport nltk\nimport random\nimport nltk\ndef perform_mutation(summary, input_text):\n    # Tokenize sentences in input text\n    sentences = nltk.sent_tokenize(input_text)\n\n    if len(sentences) == 0:\n        return input_text\n\n    if random.random() < 0.5:\n        if len(sentences) > 1:\n            sentences.pop(random.randint(0, len(sentences) - 1))\n    else:\n        # Remove common sentences\n        common_sentences = set(nltk.sent_tokenize(summary))\n        valid_sentences = [s for s in sentences if s not in common_sentences]\n\n        # If there are no valid sentences, return the original summary\n        if not valid_sentences:\n            return summary\n\n        # Calculate cosine similarity with each valid sentence\n        max_similarity = -1\n        selected_sentence = None\n        for new_sentence in valid_sentences:\n            similarity_sum = 0\n            for existing_sentence in nltk.sent_tokenize(summary):\n                similarit

In [ ]:
def perform_mutation(summary, input_text):
    # Tokenize sentences in input text
    sentences = nltk.sent_tokenize(input_text)

    if len(sentences) == 0:
        return input_text

    if random.random() < 0.4:
        if len(sentences) > 1:
            sentences.pop(random.randint(0, len(sentences) - 1))
    else:
        # Remove common sentences
        common_sentences = set(nltk.sent_tokenize(summary))
        valid_sentences = [s for s in sentences if s not in common_sentences]

        # If there are no valid sentences, return the original summary
        if not valid_sentences:
            return summary

        # Insert a random valid sentence
        new_sentence = random.choice(valid_sentences)
        summary += " " + new_sentence

    return summary


In [ ]:
!pip install tabulate

In [ ]:
from tabulate import tabulate

# Function to extract fitness score for sorting
def get_fitness(item):
    return item[1]
def genetic_algorithm(original_text, language, pop_size=30, max_iter=10):
    top_summary = original_text  # Initialize top_summary
    best_fitness = 0.0
    # Iterative loop
    for iteration in range(max_iter):
        unchanged_count = 0
        population = population_initialize(original_text, pop_size)
        selected_summaries = selection(population, original_text)
        new_population = []

        print(f"\n\nITERATION {iteration + 1}:")
        print("\nBEFORE CROSSOVER AND MUTATION:")
        for summary in selected_summaries:
            fitness = evaluate_fitness(summary, original_text)
            print(f"Summary: {summary[:50]}... Fitness: {fitness:.4f}")

        for i in range(len(selected_summaries) - 1):
            child1, child2 = crossover(selected_summaries[i], selected_summaries[i + 1])
            new_population.append(perform_mutation(child1, original_text))
            new_population.append(perform_mutation(child2, original_text))

        print("\nAFTER CROSSOVER AND MUTATION:")
        for summary in new_population:
            fitness = evaluate_fitness(summary, original_text)
            print(f"Summary: {summary[:50]}... Fitness: {fitness:.4f}")
            # Update top_summary if current summary has higher fitness
            if fitness > best_fitness:
                best_fitness = fitness
                top_summary = summary

        population = new_population

        # If population is empty, stop the genetic algorithm
        if not population:
            print("Population became empty. Stopping genetic algorithm.")
            break

        # Check if best fitness increased by 0.01 after 10 iterations
        if iteration > 10:
            if best_fitness - prev_best_fitness < 0.01:
                unchanged_count += 1
            else:
                unchanged_count = 0

            if unchanged_count >= 10:
                print("Best fitness has not improved significantly. Stopping the algorithm.")
                break

        prev_best_fitness = best_fitness

    print("Best fitness across all iterations:", best_fitness)
    return top_summary,len(nltk.sent_tokenize(original_text)),len(nltk.sent_tokenize(top_summary))


In [ ]:
!pip install sumy

In [ ]:
from nltk.util import ngrams
from nltk.tokenize import word_tokenize
from collections import Counter

def rouge_su4(original_text, extractive_reference):
    # Tokenize the original text and the extractive reference summary
    original_tokens = word_tokenize(original_text.lower())
    reference_tokens = word_tokenize(extractive_reference.lower())

    # Create 4-grams for original text and reference summary
    original_ngrams = list(ngrams(original_tokens, 4))
    reference_ngrams = list(ngrams(reference_tokens, 4))

    # Count the occurrences of each 4-gram in the original text and the reference summary
    original_counts = Counter(original_ngrams)
    reference_counts = Counter(reference_ngrams)

    # Calculate the intersection between original and reference 4-grams
    intersection = sum((original_counts & reference_counts).values())

    # Calculate the union between original and reference 4-grams
    union = sum((original_counts | reference_counts).values())

    # Calculate the ROUGE-SU4 score
    if union == 0:
        rouge_su4_score = 0.0
    else:
        rouge_su4_score = intersection / union

    return rouge_su4_score



In [ ]:
!pip install sacrebleu

In [ ]:
import pandas as pd
from rouge import Rouge
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.text_rank import TextRankSummarizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.summarizers.luhn import LuhnSummarizer

# Load the dataset
df = pd.read_csv('bbc-news-summary.csv')

# Initialize Rouge
rouge = Rouge()
language="english"
# Initialize lists to store results
data = []


from nltk.translate.bleu_score import sentence_bleu as nltk_sentence_bleu
from nltk.translate.meteor_score import meteor_score as nltk_meteor_score
from sacrebleu import sentence_bleu as sacrebleu_sentence_bleu

# Process only the first 10 rows
for idx, row in df.head(10).iterrows():
    # Extract original text and BBC summary
    original_text = row['Articles']
    bbc_summary = row['Summaries']

    # Apply TextRank using Sumy
    parser = PlaintextParser.from_string(original_text, Tokenizer("english"))
    summarizer = TextRankSummarizer()
    text_rank_summary = ' '.join([str(sentence) for sentence in summarizer(parser.document, len(original_text.split()) // 4)])
    scores = rouge.get_scores(text_rank_summary, bbc_summary)[0]
    su4=rouge_su4(text_rank_summary,bbc_summary)
    bleu_score_tr = nltk_sentence_bleu([original_text.split()], text_rank_summary.split())
    #meteor_score_tr = nltk_meteor_score(original_text, text_rank_summary)
    sacrebleu_score_tr = sacrebleu_sentence_bleu(text_rank_summary, [original_text])
    text_rank_rouge = scores['rouge-1']['f'], scores['rouge-2']['f'], scores['rouge-l']['f'],su4,bleu_score_tr,sacrebleu_score_tr

    # Apply LSA using Sumy
    lsa_summarizer = LsaSummarizer()
    lsa_summary = lsa_summarizer(parser.document, len(original_text.split()) // 4)
    lsa_summary_text = ' '.join([str(sentence) for sentence in lsa_summary])
    scores = rouge.get_scores(lsa_summary_text, bbc_summary)[0]
    su4_1=rouge_su4(lsa_summary_text,bbc_summary)
    bleu_score_lsa = nltk_sentence_bleu([original_text.split()], lsa_summary_text.split())
    #meteor_score_lsa = nltk_meteor_score(original_text, candidate)
    sacrebleu_score_lsa = sacrebleu_sentence_bleu(lsa_summary_text, [original_text])
    lsa_rouge = scores['rouge-1']['f'], scores['rouge-2']['f'], scores['rouge-l']['f'],su4_1,bleu_score_lsa,sacrebleu_score_lsa

    # Apply Luhn using Sumy
    luhn_summarizer = LuhnSummarizer()
    luhn_summary = luhn_summarizer(parser.document, len(original_text.split()) // 4)
    luhn_summary_text = ' '.join([str(sentence) for sentence in luhn_summary])
    scores = rouge.get_scores(luhn_summary_text, bbc_summary)[0]
    su4_2=rouge_su4(luhn_summary_text,bbc_summary)
    bleu_score_luhn= nltk_sentence_bleu([original_text.split()], luhn_summary_text.split())
    #meteor_score_luhn = nltk_meteor_score(original_text, luhn_summary_text)
    sacrebleu_score_luhn = sacrebleu_sentence_bleu(luhn_summary_text, [original_text])
    luhn_rouge = scores['rouge-1']['f'], scores['rouge-2']['f'], scores['rouge-l']['f'],su4_2,bleu_score_luhn,sacrebleu_score_luhn

    # Apply Genetic Algorithm
    language = "english"
    genetic_summary = genetic_algorithm(original_text, language)
    scores = rouge.get_scores(genetic_summary, bbc_summary)[0]
    su4_3=rouge_su4(genetic_summary,bbc_summary)
    bleu_score_ga = nltk_sentence_bleu([original_text.split()], genetic_summary.split())
    #meteor_score_ga = nltk_meteor_score(original_text, genetic_summary)
    sacrebleu_score_ga = sacrebleu_sentence_bleu(genetic_summary, [original_text])
    genetic_rouge = scores['rouge-1']['f'], scores['rouge-2']['f'], scores['rouge-l']['f'],su4_3,bleu_score_ga,sacrebleu_score_ga

    # Add results to data list
    data.append([original_text, bbc_summary, genetic_summary, *genetic_rouge, text_rank_summary, *text_rank_rouge, lsa_summary_text, *lsa_rouge, luhn_summary_text, *luhn_rouge])

    # Print progress message
    print(f"Row {idx+1} added.")

# Create a new DataFrame with the results
columns = ['original_text', 'bbc_summary', 'genetic_summary', 'genetic_rouge1', 'genetic_rouge2', 'genetic_rouge-l','genetic_rouge_su4','bleuga','sacrebleuga', 'text_rank_summary', 'text_rank_rouge1', 'text_rank_rouge2', 'text_rank_rouge-l','text_rank_su4','bleutr','sacrebleutr', 'lsa_summary', 'lsa_rouge1', 'lsa_rouge2', 'lsa_rouge-l','lsa_rouge_su4','bleulsa','sacrebleulsa', 'luhn_summary', 'luhn_rouge1', 'luhn_rouge2', 'luhn_rouge-l','luhn_rouge_su4','bleuluhn','sacrebleuluhn']
results_df = pd.DataFrame(data, columns=columns)




ITERATION 1:

BEFORE CROSSOVER AND MUTATION:
Summary: MPs quiz aides over royal income..Senior officials... Fitness: 0.6449
Summary: MPs quiz aides over royal income..Senior officials... Fitness: 0.6246
Summary: MPs quiz aides over royal income..Senior officials... Fitness: 0.6089
Summary: It has been reported they could be questioned abou... Fitness: 0.6088
Summary: MPs quiz aides over royal income..Senior officials... Fitness: 0.6084
Summary: MPs quiz aides over royal income..Senior officials... Fitness: 0.5984
Summary: MPs quiz aides over royal income..Senior officials... Fitness: 0.5984
Summary: MPs quiz aides over royal income..Senior officials... Fitness: 0.5889
Summary: MPs quiz aides over royal income..Senior officials... Fitness: 0.5875
Summary: MPs quiz aides over royal income..Senior officials... Fitness: 0.5676
Summary: MPs quiz aides over royal income..Senior officials... Fitness: 0.5640
Summary: Duchy officials, who will appear before the commit... Fitness: 0.5590
Summa

In [ ]:

# Create a new DataFrame with the results
columns = ['original_text', 'bbc_summary', 'genetic_summary', 'genetic_rouge1', 'genetic_rouge2', 'genetic_rouge-l','genetic_rouge_su4','bleuga','sacrebleuga', 'text_rank_summary', 'text_rank_rouge1', 'text_rank_rouge2', 'text_rank_rouge-l','text_rank_su4','bleutr','sacrebleutr', 'lsa_summary', 'lsa_rouge1', 'lsa_rouge2', 'lsa_rouge-l','lsa_rouge_su4','bleulsa','sacrebleulsa', 'luhn_summary', 'luhn_rouge1', 'luhn_rouge2', 'luhn_rouge-l','luhn_rouge_su4','bleuluhn','sacrebleuluhn']
results_df = pd.DataFrame(data, columns=columns)

# Save the DataFrame to a CSV file
results_df.to_csv('lresults.csv', index=False)
